In [1]:
import os
import sys
import numpy as np
import mongo
import time
import pickle
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
from pymatgen.io.ase import AseAtomsAdaptor


In [2]:
# docs = pickle.load(open('../cgcnn_Geometric/docs.pkl','rb'))
docs = pickle.load(open('docs_H_4k_torch.pkl','rb'))


In [3]:
len(docs)

4000

In [4]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import tqdm
from sklearn.preprocessing import StandardScaler
from cgcnn.data import *

start = time.time()

SDT = StructureDataTransformer(atom_init_loc='atom_init.json',
                              max_num_nbr=12,
                               step=0.4,
                              radius=7,
                              use_tag=False,
                              use_fixed_info=True,
                              is_initial=True,
                              use_distance=True,
#                               bond_property=True,
                              )


SDT_out = SDT.transform(docs)

structures = SDT_out[0]

#Settings necessary to build the model (since they are size of vectors as inputs)
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]

import multiprocess as mp

with mp.Pool(4) as pool:
    SDT_list = list(tqdm.tqdm(pool.imap(lambda x: SDT_out[x],range(len(SDT_out)),chunksize=40),total=len(SDT_out)))
    
structures = SDT_list[0]
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]

100%|██████████| 4000/4000 [06:07<00:00, 10.88it/s] 


In [5]:
target_list = np.array([doc['energy'] for doc in docs]).reshape(-1,1)
SDT = []
for i, sdt in enumerate(SDT_list):
    SDT.append(sdt+tuple(torch.tensor(target_list[i]).view(-1)))
SDT_list = SDT

In [6]:
pickle.dump(SDT_list, open('./SDT_list_H.pkl','wb'))

In [7]:
SDT_list = pickle.load(open('./SDT_list_H.pkl', 'rb'))

In [8]:
from geometric import Convert


converter = Convert('./SDT_list_H.pkl', './SDT_list_H.pt')
data = converter.geometric()

0it [00:00, ?it/s]

11


RuntimeError: expected a non-empty list of Tensors

In [10]:
SDT_list[11][2]

tensor([[99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99]])

In [22]:
data[0]

Data(connectivity=[391, 1], edge_attr=[4692, 19], edge_index=[2, 4692], graph_idx=[10], x=[391, 99], y=[10])

In [14]:
edge_attr = []
edge_attr.append(torch.tensor([]))
edge_attr.append(torch.tensor([2,3,]))
edge_attr = torch.stack(edge_attr)#.float()


RuntimeError: Expected object of scalar type Float but got scalar type Long for sequence element 1 in sequence argument at position #1 'tensors'

In [15]:
49*12

588